In [2]:
import os
import cv2 #pip install opencv-[ython]
import pandas as pd
import numpy as np

Welcome, today I am going to try and classify some images I found on kaggle to Identify landscape

If you wish to follow along you can download the data here

https://www.kaggle.com/datasets/utkarshsaxenadn/landscape-recognition-image-dataset-12k-images

## Section 1: Looking at Training Images

In [ ]:
training_path = r'C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Training Data'
testing_path = r'C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Testing Data'
val_path = r'C:\Users\brunolopez\mldata\landscape_recognition\Landscape Classification\Landscape Classification\Validation Data'

